## Making Transformers Efficient in Production

### Intent Detection as a Case Study

In [1]:
from transformers import pipeline

bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=bert_ckpt)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [2]:
query = """Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th in
Paris and I need a 15 passenger van"""

pipe(query)

[{'label': 'car_rental', 'score': 0.5490034818649292}]

### Creating a Performance Benchmark

In [3]:
class PerformanceBenchmark:
    def __init__(self, pipeline, dataset, optim_type="BERT baseline"):
        self.pipeline = pipeline
        self.dataset = dataset
        self.optim_type = optim_type
    
    def compute_accuracy(self):
        # We'll define this later
        pass
    
    def compute_size(self):
        # We'll define this later
        pass
    
    def time_pipeline(self):
        # We'll define this later
        pass
    
    def run_benchmark(self):
        metrics = {}
        metrics[self.optim_type] = self.compute_size()
        metrics[self.optim_type].update(self.time_pipeline())
        metrics[self.optim_type].update(self.compute_accuracy())
        return metrics

In [4]:
from datasets import load_dataset

clinc = load_dataset("clinc_oos", "plus")

In [5]:
sample = clinc["test"][42]
sample

{'text': 'transfer $100 from my checking to saving account', 'intent': 133}

In [6]:
intents = clinc["test"].features["intent"]
intents.int2str(sample["intent"])

'transfer'

In [7]:
from datasets import load_metric

accuracy_score = load_metric("accuracy")

C:\Users\Aquib\AppData\Local\Temp\ipykernel_17732\3510557904.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_score = load_metric("accuracy")


In [8]:
def compute_accuracy(self):
    """This overrides the PerformanceBenchmark.compute_accuracy() method"""
    preds, labels = [], []
    for example in self.dataset:
        pred = self.pipeline(example["text"])[0]["label"]
        label = example["intent"]
        preds.append(intents.str2int(pred))
        labels.append(label)
    accuracy = accuracy_score.compute(predictions=preds, references=labels)
    print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")
    return accuracy

In [9]:
PerformanceBenchmark.compute_accuracy = compute_accuracy

In [10]:
list(pipe.model.state_dict().items())[42]

('bert.encoder.layer.2.attention.self.value.bias',
 tensor([-2.7834e-02,  4.9434e-02,  8.3551e-02,  4.1092e-02,  6.0157e-01,
          1.1774e-01, -5.2112e-02, -6.5143e-02, -2.9358e-02, -4.2250e-02,
          7.9177e-02,  8.0409e-02,  2.9921e-03,  1.7816e-01, -5.0480e-02,
         -1.5634e-01, -2.1707e-02,  1.4381e-02,  2.5132e-02, -2.4110e-02,
         -1.9183e-01, -7.8657e-02,  5.0709e-02,  3.3632e-02, -3.1946e-02,
          1.1616e-01,  9.2720e-02, -1.1787e-01,  2.3233e-01, -1.2678e-02,
         -1.3138e-01, -4.0024e-02,  7.4823e-02, -5.4148e-02, -1.5184e-01,
         -7.4407e-02,  1.1559e-01,  8.2729e-02, -1.3787e-01,  8.3528e-02,
          1.2154e-01,  1.6880e-02, -5.6629e-02, -3.9295e-02,  5.3725e-02,
          6.8602e-02, -1.1294e-01,  4.4001e-02, -2.5884e-01,  1.6767e-01,
          1.8316e-01,  5.6272e-02, -3.6874e-02, -2.7938e-02, -9.3204e-02,
         -7.5239e-03,  4.1141e-02, -1.1542e-02, -9.9749e-02, -3.0910e-02,
          4.1398e-02, -4.4389e-02, -2.6279e-02,  7.2100e-02, 

In [11]:
import torch 

torch.save(pipe.model.state_dict(), "model.pt")

In [12]:
import torch
from pathlib import Path

def compute_size(self):
    """This overrides the PerformanceBenchmark.compute_size() method"""
    state_dict = self.pipeline.model.state_dict()
    tmp_path = Path("model.pt")
    torch.save(state_dict, tmp_path)
    # Calculate size in megabytes
    size_mb = Path(tmp_path).stat().st_size / (1024 * 1024)
    # Delete temporary file
    tmp_path.unlink()
    print(f"Model size (MB) - {size_mb:.2f}")
    return {"size_mb": size_mb}

PerformanceBenchmark.compute_size = compute_size

In [13]:
from time import perf_counter
for _ in range(3):
    start_time = perf_counter()
    _ = pipe(query)
    latency = perf_counter() - start_time
    print(f"Latency (ms) - {1000 * latency:.3f}")

Latency (ms) - 50.613
Latency (ms) - 51.883
Latency (ms) - 47.774


In [14]:
import numpy as np

def time_pipeline(self, query="What is the pin number for my account?"):
    """This overrides the PerformanceBenchmark.time_pipeline() method"""
    latencies = []
    # Warmup
    for _ in range(10):
        _ = self.pipeline(query)
    # Timed run
    for _ in range(100):
        start_time = perf_counter()
        _ = self.pipeline(query)
        latency = perf_counter() - start_time
        latencies.append(latency)
    # Compute run statistics
    time_avg_ms = 1000 * np.mean(latencies)
    time_std_ms = 1000 * np.std(latencies)
    print(f"Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f}")
    return {"time_avg_ms": time_avg_ms, "time_std_ms": time_std_ms}

<>:18: SyntaxWarning: invalid escape sequence '\-'
<>:18: SyntaxWarning: invalid escape sequence '\-'
C:\Users\Aquib\AppData\Local\Temp\ipykernel_17732\2155140460.py:18: SyntaxWarning: invalid escape sequence '\-'
  print(f"Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f}")


In [15]:
PerformanceBenchmark.time_pipeline = time_pipeline

In [16]:
pb = PerformanceBenchmark(pipe, clinc["test"])
perf_metrics = pb.run_benchmark()

Model size (MB) - 418.15
Average latency (ms) - 37.57 +\- 3.24
Accuracy on test set - 0.867



### Making Models Smaller via Knowledge Distillation

#### Creating a Knowledge Distillation Trainer

In [17]:
from transformers import TrainingArguments

class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature

In [18]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs_stu = model(**inputs)
        # Extract cross-entropy loss and logits from student
        loss_ce = outputs_stu.loss
        logits_stu = outputs_stu.logits
        # Extract logits from teacher
        with torch.no_grad():
            outputs_tea = self.teacher_model(**inputs)
            logits_tea = outputs_tea.logits
        # Soften probabilities and compute distillation loss
        loss_fct = nn.KLDivLoss(reduction="batchmean")
        loss_kd = self.args.temperature ** 2 * loss_fct(
            F.log_softmax(logits_stu / self.args.temperature, dim=-1),
            F.softmax(logits_tea / self.args.temperature, dim=-1))
        # Return weighted student loss
        loss = self.args.alpha * loss_ce + (1. - self.args.alpha) * loss_kd
        return (loss, outputs_stu) if return_outputs else loss

#### Choosing a Good Student Initialization

In [19]:
from transformers import AutoTokenizer

student_ckpt = "distilbert-base-uncased"
student_tokenizer = AutoTokenizer.from_pretrained(student_ckpt)

def tokenize_text(batch):
    return student_tokenizer(batch["text"], truncation=True)

clinc_enc = clinc.map(tokenize_text, batched=True, remove_columns=["text"])
clinc_enc = clinc_enc.rename_column("intent", "labels")

Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [20]:
def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_score.compute(predictions=predictions, references=labels)

In [21]:
batch_size = 48

finetuned_ckpt = "distilbert-base-uncased-finetuned-clinc"
student_training_args = DistillationTrainingArguments(
    output_dir=finetuned_ckpt, evaluation_strategy = "epoch",
    num_train_epochs=5, learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, alpha=1, weight_decay=0.01,
    push_to_hub=True)

C:\Users\Aquib\AppData\Local\anaconda3\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

In [23]:
from transformers import AutoConfig

num_labels = intents.num_classes
student_config = (AutoConfig
                 .from_pretrained(student_ckpt, num_labels=num_labels,
                 id2label=id2label, label2id=label2id))

In [24]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def student_init():
    return (AutoModelForSequenceClassification
            .from_pretrained(student_ckpt, config=student_config).to(device))

In [25]:
teacher_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
teacher_model = (AutoModelForSequenceClassification
                 .from_pretrained(teacher_ckpt, num_labels=num_labels)
                 .to(device))

distilbert_trainer = DistillationTrainer(model_init=student_init,
    teacher_model=teacher_model, args=student_training_args,
    train_dataset=clinc_enc['train'], eval_dataset=clinc_enc['validation'],
    compute_metrics=compute_metrics, tokenizer=student_tokenizer)

distilbert_trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Aquib\AppData\Local\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_d

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.292810,0.726774
2,3.798800,1.886249,0.851613
3,3.798800,1.174241,0.887742
4,1.713400,0.879092,0.907097
5,0.919400,0.792140,0.914516


TrainOutput(global_step=1590, training_loss=2.0666040432528128, metrics={'train_runtime': 238.5104, 'train_samples_per_second': 319.693, 'train_steps_per_second': 6.666, 'total_flos': 414689637990180.0, 'train_loss': 2.0666040432528128, 'epoch': 5.0})

In [26]:
distilbert_trainer.push_to_hub("Training completed!")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SkillRipper/distilbert-base-uncased-finetuned-clinc/commit/5befb28d26af986884856ef80fdb9ebfea181a6c', commit_message='Training completed!', commit_description='', oid='5befb28d26af986884856ef80fdb9ebfea181a6c', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
# finetuned_ckpt = "transformersbook/distilbert-base-uncased-finetuned-clinc"
finetuned_ckpt = "SkillRipper/distilbert-base-uncased-finetuned-clinc"

pipe = pipeline("text-classification", model=finetuned_ckpt)

config.json:   0%|          | 0.00/8.54k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [28]:
optim_type = "DistilBERT"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())

Model size (MB) - 255.88
Average latency (ms) - 14.57 +\- 0.57
Accuracy on test set - 0.854


In [29]:
import pandas as pd

def plot_metrics(perf_metrics, current_optim_type):
    df = pd.DataFrame.from_dict(perf_metrics, orient='index')
    
    for idx in df.index:
        df_opt = df.loc[idx]
        # Add a dashed circle around the current optimization type
        if idx == current_optim_type:
            plt.scatter(df_opt["time_avg_ms"], df_opt["accuracy"] * 100,
                        alpha=0.5, s=df_opt["size_mb"], label=idx,
                        marker='$\u25CC$')
        else:
            plt.scatter(df_opt["time_avg_ms"], df_opt["accuracy"] * 100,
                        s=df_opt["size_mb"], label=idx, alpha=0.5)
    
    legend = plt.legend(bbox_to_anchor=(1,1))
    for handle in legend.legendHandles:
        handle.set_sizes([20])
    
    plt.ylim(80,90)
    # Use the slowest model to define the x-axis range
    xlim = int(perf_metrics["BERT baseline"]["time_avg_ms"] + 3)
    plt.xlim(1, xlim)
    plt.ylabel("Accuracy (%)")
    plt.xlabel("Average latency (ms)")
    plt.show()

### Finding Good Hyperparameters with Optuna

In [31]:
def objective(trial):
    x = trial.suggest_float("x", -2, 2)
    y = trial.suggest_float("y", -2, 2)
    return (1 - x) ** 2 + 100 * (y - x ** 2) ** 2

In [36]:
import optuna

study = optuna.create_study()
study.optimize(objective, n_trials=1000)

[I 2024-09-19 15:26:46,944] A new study created in memory with name: no-name-36b7188a-ca9d-419d-8a77-1d675a2578e1
[I 2024-09-19 15:26:46,948] Trial 0 finished with value: 407.59484307150524 and parameters: {'x': 1.93971281656703, 'y': 1.7457761566547214}. Best is trial 0 with value: 407.59484307150524.
[I 2024-09-19 15:26:46,949] Trial 1 finished with value: 685.9564209179578 and parameters: {'x': -1.5499477880735268, 'y': -0.20429605795784234}. Best is trial 0 with value: 407.59484307150524.
[I 2024-09-19 15:26:46,950] Trial 2 finished with value: 762.1737838095153 and parameters: {'x': -1.0088363378990381, 'y': -1.7356804852162147}. Best is trial 0 with value: 407.59484307150524.
[I 2024-09-19 15:26:46,951] Trial 3 finished with value: 2.8260157475521894 and parameters: {'x': -0.5843346063667703, 'y': 0.28524198556311786}. Best is trial 3 with value: 2.8260157475521894.
[I 2024-09-19 15:26:46,952] Trial 4 finished with value: 1690.499113313176 and parameters: {'x': 1.467530949111099,

[I 2024-09-19 15:26:47,237] Trial 42 finished with value: 14.319564273798427 and parameters: {'x': -0.06675624258800544, 'y': -0.358608273145629}. Best is trial 33 with value: 1.9438996505856836.
[I 2024-09-19 15:26:47,246] Trial 43 finished with value: 789.714340764756 and parameters: {'x': 1.6647518498138032, 'y': -0.03800058398552414}. Best is trial 33 with value: 1.9438996505856836.
[I 2024-09-19 15:26:47,254] Trial 44 finished with value: 53.851093188888754 and parameters: {'x': 0.17469927613356684, 'y': 0.7596972633920499}. Best is trial 33 with value: 1.9438996505856836.
[I 2024-09-19 15:26:47,262] Trial 45 finished with value: 2.3803794816839083 and parameters: {'x': 0.5835053988331442, 'y': 0.1919217691047851}. Best is trial 33 with value: 1.9438996505856836.
[I 2024-09-19 15:26:47,271] Trial 46 finished with value: 260.24886349855416 and parameters: {'x': 1.020258042879982, 'y': -0.5722953120247168}. Best is trial 33 with value: 1.9438996505856836.
[I 2024-09-19 15:26:47,279]

[I 2024-09-19 15:26:47,620] Trial 84 finished with value: 132.53293384616595 and parameters: {'x': 0.5607598267227767, 'y': 1.4648428293922116}. Best is trial 77 with value: 0.47494546485194655.
[I 2024-09-19 15:26:47,629] Trial 85 finished with value: 74.20156204341941 and parameters: {'x': 0.899231428964042, 'y': 1.6699615059336579}. Best is trial 77 with value: 0.47494546485194655.
[I 2024-09-19 15:26:47,638] Trial 86 finished with value: 1.8640274933522898 and parameters: {'x': 1.0854094070986737, 'y': 1.3143755612817318}. Best is trial 77 with value: 0.47494546485194655.
[I 2024-09-19 15:26:47,647] Trial 87 finished with value: 197.69579979317413 and parameters: {'x': 1.6266139648125233, 'y': 1.241226576893423}. Best is trial 77 with value: 0.47494546485194655.
[I 2024-09-19 15:26:47,656] Trial 88 finished with value: 98.8025315710884 and parameters: {'x': 1.03342192819534, 'y': 0.07397187496612723}. Best is trial 77 with value: 0.47494546485194655.
[I 2024-09-19 15:26:47,665] Tri

[I 2024-09-19 15:26:48,008] Trial 126 finished with value: 0.12460369501121005 and parameters: {'x': 0.6495794753782169, 'y': 0.4177000833371404}. Best is trial 126 with value: 0.12460369501121005.
[I 2024-09-19 15:26:48,016] Trial 127 finished with value: 1.5743103395290463 and parameters: {'x': 0.7018264531919487, 'y': 0.6144374755310591}. Best is trial 126 with value: 0.12460369501121005.
[I 2024-09-19 15:26:48,025] Trial 128 finished with value: 4.2322631566831355 and parameters: {'x': 0.6627508852349364, 'y': 0.6421802592620031}. Best is trial 126 with value: 0.12460369501121005.
[I 2024-09-19 15:26:48,033] Trial 129 finished with value: 11.375958787703508 and parameters: {'x': 0.4336143985439375, 'y': 0.5205145463750781}. Best is trial 126 with value: 0.12460369501121005.
[I 2024-09-19 15:26:48,043] Trial 130 finished with value: 0.2950703670213121 and parameters: {'x': 0.6377823778502187, 'y': 0.4472470681632175}. Best is trial 126 with value: 0.12460369501121005.
[I 2024-09-19 

[I 2024-09-19 15:26:48,357] Trial 168 finished with value: 13.846601575857092 and parameters: {'x': 0.7091796992255832, 'y': 0.8739078815122124}. Best is trial 149 with value: 0.0026389185181298733.
[I 2024-09-19 15:26:48,366] Trial 169 finished with value: 9.857943358518742 and parameters: {'x': 1.0045268855911742, 'y': 0.6951009709694737}. Best is trial 149 with value: 0.0026389185181298733.
[I 2024-09-19 15:26:48,374] Trial 170 finished with value: 1.8570185519330678 and parameters: {'x': 0.8251191819792943, 'y': 0.8159673359295898}. Best is trial 149 with value: 0.0026389185181298733.
[I 2024-09-19 15:26:48,383] Trial 171 finished with value: 0.36276855415906 and parameters: {'x': 0.9228896241026721, 'y': 0.9114598834185812}. Best is trial 149 with value: 0.0026389185181298733.
[I 2024-09-19 15:26:48,392] Trial 172 finished with value: 6.611468010365749 and parameters: {'x': 0.9375100216952176, 'y': 1.1359768451487717}. Best is trial 149 with value: 0.0026389185181298733.
[I 2024-0

...

[I 2024-09-19 15:26:55,766] Trial 798 finished with value: 6.648043989271654 and parameters: {'x': 1.2021506665638544, 'y': 1.188121888530306}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:55,779] Trial 799 finished with value: 0.07590981134300731 and parameters: {'x': 1.1004578079960043, 'y': 1.1853523603511553}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:55,792] Trial 800 finished with value: 4.675062509155056 and parameters: {'x': 1.04137446534016, 'y': 1.3006401161024057}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:55,805] Trial 801 finished with value: 205.70175355260506 and parameters: {'x': -0.08666018134831105, 'y': 1.4376181060309912}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:55,818] Trial 802 finished with value: 3.3637779133820542 and parameters: {'x': 1.1648282333087885, 'y': 1.1741609259629568}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-1

[I 2024-09-19 15:26:56,351] Trial 840 finished with value: 296.8452990311518 and parameters: {'x': 0.9162493747556801, 'y': -0.8833866301398019}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:56,369] Trial 841 finished with value: 2.0010226952986163 and parameters: {'x': 0.9997507399731527, 'y': 1.1409590492360149}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:56,386] Trial 842 finished with value: 17.245066576391938 and parameters: {'x': 1.1167450951802989, 'y': 0.8320119449584278}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:56,403] Trial 843 finished with value: 13.170274015567257 and parameters: {'x': 0.9131899800426029, 'y': 1.1967208188346308}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:56,418] Trial 844 finished with value: 1.6424766296352338 and parameters: {'x': 1.023357281050844, 'y': 0.919122266610541}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-1

[I 2024-09-19 15:26:56,964] Trial 882 finished with value: 507.80150716486776 and parameters: {'x': 1.855156782236607, 'y': 1.1897847264812231}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:56,978] Trial 883 finished with value: 1.9525426819861174 and parameters: {'x': 1.0617183128630538, 'y': 0.9876487296305537}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:56,991] Trial 884 finished with value: 680.449828340582 and parameters: {'x': 1.1357477127399902, 'y': -1.318585137400477}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:57,005] Trial 885 finished with value: 226.27909731545319 and parameters: {'x': 1.6276611635245162, 'y': 1.1463332966979793}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:57,019] Trial 886 finished with value: 2.5232557190909572 and parameters: {'x': 0.9481186720494091, 'y': 1.0576918597660752}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19

[I 2024-09-19 15:26:57,582] Trial 924 finished with value: 0.511918464335044 and parameters: {'x': 0.8375509765127463, 'y': 0.771171533428378}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:57,597] Trial 925 finished with value: 0.7784503638894703 and parameters: {'x': 0.880702474833817, 'y': 0.8630564387667109}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:57,611] Trial 926 finished with value: 7.953236786791593 and parameters: {'x': 0.7771977033244779, 'y': 0.8851696178326416}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:57,627] Trial 927 finished with value: 0.9695180537011531 and parameters: {'x': 0.9402301586054976, 'y': 0.7857502186360839}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:57,642] Trial 928 finished with value: 0.20375947862015883 and parameters: {'x': 0.9460837054707598, 'y': 0.8502578058441623}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19

[I 2024-09-19 15:26:58,206] Trial 966 finished with value: 1.4591442294018488 and parameters: {'x': 0.916734727369325, 'y': 0.9609102830956796}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:58,220] Trial 967 finished with value: 10.947703501905972 and parameters: {'x': 0.8440845788557969, 'y': 1.0429843569012156}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:58,235] Trial 968 finished with value: 17.806810085444152 and parameters: {'x': 1.1053057541719304, 'y': 0.799851064637535}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:58,249] Trial 969 finished with value: 3.0353000516265682 and parameters: {'x': 1.0396102676831889, 'y': 0.9066134173646745}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-19 15:26:58,264] Trial 970 finished with value: 1.0261252075884328 and parameters: {'x': 0.9479918813079404, 'y': 0.9998528479006221}. Best is trial 493 with value: 0.000442502037418168.
[I 2024-09-1

In [ ]:
study.best_params

In [33]:
def hp_space(trial):

    return {"num_train_epochs": trial.suggest_int("num_train_epochs", 5, 10),
        "alpha": trial.suggest_float("alpha", 0, 1),
        "temperature": trial.suggest_int("temperature", 2, 20)}

In [ ]:
best_run = distilbert_trainer.hyperparameter_search(
    n_trials=20, direction="maximize", hp_space=hp_space)

[I 2024-09-19 15:26:58,698] A new study created in memory with name: no-name-950748a5-06af-4420-a33e-12546db91619
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.410439,0.672581
2,0.644900,0.142742,0.842258
3,0.644900,0.074758,0.894839
4,0.166900,0.053175,0.910323
5,0.084500,0.044639,0.923226
6,0.084500,0.040241,0.922581
7,0.065000,0.037843,0.925806
8,0.057700,0.037051,0.927742


[I 2024-09-19 15:32:45,149] Trial 0 finished with value: 0.927741935483871 and parameters: {'num_train_epochs': 8, 'alpha': 0.3135032537062875, 'temperature': 2}. Best is trial 0 with value: 0.927741935483871.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.197174,0.569677
2,0.312600,0.100628,0.811935
3,0.312600,0.070571,0.871613
4,0.115900,0.057442,0.887097
5,0.079600,0.050582,0.892581
6,0.079600,0.048641,0.895161


[I 2024-09-19 15:37:10,449] Trial 1 finished with value: 0.8951612903225806 and parameters: {'num_train_epochs': 6, 'alpha': 0.979315817401237, 'temperature': 17}. Best is trial 0 with value: 0.927741935483871.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.192412,0.578710
2,0.308800,0.095389,0.822258
3,0.308800,0.064737,0.877419
4,0.110000,0.050768,0.891613
5,0.071800,0.042599,0.904194
6,0.071800,0.037887,0.908065
7,0.058000,0.035747,0.911613
8,0.052100,0.034889,0.913871


[I 2024-09-19 15:49:53,296] Trial 3 finished with value: 0.9138709677419354 and parameters: {'num_train_epochs': 8, 'alpha': 0.7609341909546662, 'temperature': 18}. Best is trial 0 with value: 0.927741935483871.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.201887,0.588065
2,0.322300,0.100059,0.822258
3,0.322300,0.068092,0.876774
4,0.115600,0.053779,0.889355
5,0.076300,0.045709,0.900323
6,0.076300,0.041688,0.904839
7,0.063000,0.040540,0.903871


[I 2024-09-19 15:55:28,991] Trial 4 finished with value: 0.9038709677419355 and parameters: {'num_train_epochs': 7, 'alpha': 0.8418102774144052, 'temperature': 11}. Best is trial 0 with value: 0.927741935483871.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.421135,0.667419


In [ ]:
print(best_run)

In [ ]:
for k,v in best_run.hyperparameters.items():
    setattr(student_training_args, k, v)

# Define a new repository to store our distilled model
distilled_ckpt = "distilbert-base-uncased-distilled-clinc"
student_training_args.output_dir = distilled_ckpt

# Create a new Trainer with optimal parameters
distil_trainer = DistillationTrainer(model_init=student_init,
    teacher_model=teacher_model, args=student_training_args,
    train_dataset=clinc_enc['train'], eval_dataset=clinc_enc['validation'],
    compute_metrics=compute_metrics, tokenizer=student_tokenizer)

distil_trainer.train();

In [ ]:
# distil_trainer.push_to_hub("Training complete")

### Benchmarking Our Distilled Model

In [ ]:
distilled_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
pipe = pipeline("text-classification", model=distilled_ckpt)
optim_type = "Distillation"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())

In [ ]:
plot_metrics(perf_metrics, optim_type)

### Making Models Faster with Quantization

In [ ]:
import matplotlib.pyplot as plt

state_dict = pipe.model.state_dict()
weights = state_dict["distilbert.transformer.layer.0.attention.out_lin.weight"]
plt.hist(weights.flatten().numpy(), bins=250, range=(-0.3,0.3), edgecolor="C0")
plt.show()

In [ ]:
zero_point = 0
scale = (weights.max() - weights.min()) / (127 - (-128))

In [ ]:
(weights / scale + zero_point).clamp(-128, 127).round().char()

In [ ]:
from torch import quantize_per_tensor

dtype = torch.qint8
quantized_weights = quantize_per_tensor(weights, scale, zero_point, dtype)
quantized_weights.int_repr()

In [ ]:
%%timeit
weights @ weights

In [ ]:
from torch.nn.quantized import QFunctional

q_fn = QFunctional()

In [ ]:
%%timeit
q_fn.mul(quantized_weights, quantized_weights)

In [ ]:
import sys

sys.getsizeof(weights.storage()) / sys.getsizeof(quantized_weights.storage())

In [ ]:
from torch.quantization import quantize_dynamic

model_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt).to("cpu"))

model_quantized = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

In [ ]:
pipe = pipeline("text-classification", model=model_quantized,
                 tokenizer=tokenizer)
optim_type = "Distillation + quantization"
pb = PerformanceBenchmark(pipe, clinc["test"], optim_type=optim_type)
perf_metrics.update(pb.run_benchmark())

### Optimizing Inference with ONNX and the ONNX Runtime

In [ ]:
import os
from psutil import cpu_count

os.environ["OMP_NUM_THREADS"] = f"{cpu_count()}"
os.environ["OMP_WAIT_POLICY"] = "ACTIVE"

In [ ]:
from transformers.convert_graph_to_onnx import convert

model_ckpt = "transformersbook/distilbert-base-uncased-distilled-clinc"
onnx_model_path = Path("onnx/model.onnx")
convert(framework="pt", model=model_ckpt, tokenizer=tokenizer,
     output=onnx_model_path, opset=12, pipeline_name="text-classification")

In [ ]:
from onnxruntime import (GraphOptimizationLevel, InferenceSession,
                         SessionOptions)

def create_model_for_provider(model_path, provider="CPUExecutionProvider"):
    options = SessionOptions()
    options.intra_op_num_threads = 1
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
    session = InferenceSession(str(model_path), options, providers=[provider])
    session.disable_fallback()
    return session

onnx_model = create_model_for_provider(onnx_model_path)

In [ ]:
inputs = clinc_enc["test"][:1]
del inputs["labels"]
logits_onnx = onnx_model.run(None, inputs)[0]
logits_onnx.shape

In [ ]:
np.argmax(logits_onnx)

In [ ]:
clinc_enc["test"][0]["labels"]

In [ ]:
from scipy.special import softmax

class OnnxPipeline:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __call__(self, query):
        model_inputs = self.tokenizer(query, return_tensors="pt")
        inputs_onnx = {k: v.cpu().detach().numpy()
        for k, v in model_inputs.items()}
        logits = self.model.run(None, inputs_onnx)[0][0, :]
        probs = softmax(logits)
        pred_idx = np.argmax(probs).item()
        return [{"label": intents.int2str(pred_idx), "score": probs[pred_idx]}]

In [ ]:
pipe = OnnxPipeline(onnx_model, tokenizer)
pipe(query)

In [ ]:
class OnnxPerformanceBenchmark(PerformanceBenchmark):
    def __init__(self, *args, model_path, **kwargs):
        super().__init__(*args, **kwargs)
        self.model_path = model_path

    def compute_size(self):
        size_mb = Path(self.model_path).stat().st_size / (1024 * 1024)
        print(f"Model size (MB) - {size_mb:.2f}")
        return {"size_mb": size_mb}

In [ ]:
optim_type = "Distillation + ORT"
pb = OnnxPerformanceBenchmark(pipe, clinc["test"], optim_type,
                              model_path="onnx/model.onnx")
perf_metrics.update(pb.run_benchmark())

In [ ]:
plot_metrics(perf_metrics, optim_type)

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

model_input = "onnx/model.onnx"
model_output = "onnx/model.quant.onnx"
quantize_dynamic(model_input, model_output, weight_type=QuantType.QInt8)

In [ ]:
onnx_quantized_model = create_model_for_provider(model_output)
pipe = OnnxPipeline(onnx_quantized_model, tokenizer)
optim_type = "Distillation + ORT (quantized)"
pb = OnnxPerformanceBenchmark(pipe, clinc["test"], optim_type,
                             model_path=model_output)
perf_metrics.update(pb.run_benchmark())